##### Авторские права 2019 Авторы TensorFlow.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Сохраните и загрузите модель, используя стратегию распространения

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Запускаем в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Посмотреть исходный код на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Скачать блокнот</a></td>
</table>

## Обзор

It's common to save and load a model during training. There are two sets of APIs for saving and loading a keras model: a high-level API, and a low-level API. This tutorial demonstrates how you can use the SavedModel APIs when using `tf.distribute.Strategy`. To learn about SavedModel and serialization in general, please read the [saved model guide](../../guide/saved_model.ipynb), and the [Keras model serialization guide](../../guide/keras/save_and_serialize.ipynb). Let's start with a simple example: 

Зависимости импорта:

In [ ]:
import tensorflow_datasets as tfds

import tensorflow as tf


Подготовьте данные и модель с помощью `tf.distribute.Strategy` :

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
  datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000

  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

  train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

  return train_dataset, eval_dataset

def get_model():
  with mirrored_strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

Обучите модель: 

In [ ]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

## Сохраните и загрузите модель

Теперь, когда у вас есть простая модель для работы, давайте взглянем на API сохранения / загрузки. Доступны два набора API:

- Высокоуровневые keras `model.save` и `tf.keras.models.load_model`
- Низкий уровень `tf.saved_model.save` и `tf.saved_model.load`


### API Keras

Вот пример сохранения и загрузки модели с помощью API Keras:

In [ ]:
keras_model_path = "/tmp/keras_save"
model.save(keras_model_path)

Восстановить модель без `tf.distribute.Strategy` :

In [ ]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

После восстановления модели вы можете продолжить обучение на ней, даже без `compile()` , поскольку она уже скомпилирована перед сохранением. Модель сохраняется в стандартном прото-формате `SavedModel` Для получения дополнительной информации, пожалуйста, обратитесь к [руководству по формату `saved_model`](../../guide/saved_model.ipynb) .

Теперь загрузим модель и `tf.distribute.Strategy` ее с помощью tf.distribute.Strategy:

In [ ]:
another_strategy = tf.distribute.OneDeviceStrategy("/cpu:0")
with another_strategy.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

Как видите, загрузка работает `tf.distribute.Strategy` с tf.distribute.Strategy. Используемая здесь стратегия не обязательно должна совпадать со стратегией, использованной до сохранения. 

### API `tf.saved_model`

Теперь давайте посмотрим на API нижнего уровня. Сохранение модели аналогично keras API:

In [ ]:
model = get_model()  # get a fresh model
saved_model_path = "/tmp/tf_save"
tf.saved_model.save(model, saved_model_path)

Загрузка может быть произведена с помощью `tf.saved_model.load()` . Однако, поскольку это API нижнего уровня (и, следовательно, имеет более широкий диапазон вариантов использования), он не возвращает модель Keras. Вместо этого он возвращает объект, содержащий функции, которые можно использовать для вывода. Например:

In [ ]:
DEFAULT_FUNCTION_KEY = "serving_default"
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

Загруженный объект может содержать несколько функций, каждая из которых связана с ключом. `"serving_default"` - это ключ по умолчанию для функции вывода с сохраненной моделью Keras. Чтобы сделать вывод с помощью этой функции: 

In [ ]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

Вы также можете загружать и выполнять логический вывод распределенным образом:

In [ ]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

  dist_predict_dataset = another_strategy.experimental_distribute_dataset(
      predict_dataset)

  # Calling the function in a distributed manner
  for batch in dist_predict_dataset:
    another_strategy.run(inference_func,args=(batch,))

Вызов восстановленной функции - это просто прямой переход к сохраненной модели (прогноз). Что делать, если вы не хотите продолжать обучение загруженной функции? Или встроить загруженную функцию в более крупную модель? Обычной практикой является перенос этого загруженного объекта в слой Keras для достижения этой цели. К счастью, в [TF Hub](https://www.tensorflow.org/hub) для этой цели есть [hub.KerasLayer](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/keras_layer.py) , показанный здесь:

In [ ]:
import tensorflow_hub as hub

def build_model(loaded):
  x = tf.keras.layers.Input(shape=(28, 28, 1), name='input_x')
  # Wrap what's loaded to a KerasLayer
  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
  model = tf.keras.Model(x, keras_layer)
  return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_model(loaded)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])
  model.fit(train_dataset, epochs=2)

Как видите, `hub.KerasLayer` оборачивает результат, загруженный обратно из `tf.saved_model.load()` в слой Keras, который можно использовать для построения другой модели. Это очень полезно для трансферного обучения. 

### Какой API мне следует использовать?

Для экономии, если вы работаете с моделью keras, почти всегда рекомендуется использовать API `model.save()` Если то, что вы сохраняете, не является моделью Keras, тогда ваш единственный выбор - API нижнего уровня.

Какой API вы используете для загрузки, зависит от того, что вы хотите получить от API загрузки. Если вы не можете (или не хотите) получить модель `tf.saved_model.load()` . В противном случае используйте `tf.keras.models.load_model()` . Обратите внимание, что вы можете вернуть модель Keras, только если вы сохранили модель Keras.

Можно смешивать и сопоставлять API. Вы можете сохранить модель `model.save` с помощью model.save и загрузить модель, отличную от Keras, с помощью низкоуровневого API `tf.saved_model.load` . 

In [ ]:
model = get_model()

# Saving the model using Keras's save() API
model.save(keras_model_path) 

another_strategy = tf.distribute.MirroredStrategy()
# Loading the model using lower level API
with another_strategy.scope():
  loaded = tf.saved_model.load(keras_model_path)

### Сохранение / загрузка с локального устройства

При сохранении и загрузке с локального устройства io во время удаленной работы, например, с использованием облачного TPU, `experimental_io_device` , чтобы установить для устройства io значение localhost.

In [ ]:
model = get_model()

# Saving the model to a path on localhost.
saved_model_path = "/tmp/tf_save"
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.save(saved_model_path, options=save_options)

# Loading the model from a path on localhost.
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
  loaded = tf.keras.models.load_model(saved_model_path, options=load_options)

### Предостережения

Особый случай - это когда у вас есть модель Keras, которая не имеет четко определенных входных данных. Например, последовательная модель может быть создана без каких-либо входных форм ( `Sequential([Dense(3), ...]` ). Подклассовые модели также не имеют четко определенных входных данных после инициализации. В этом случае вам следует придерживаться API нижнего уровня как при сохранении, так и при загрузке, иначе вы получите ошибку.

Чтобы проверить, имеет ли ваша модель четко определенные входы, просто проверьте, имеет ли `model.inputs` `None` . Если это не `None` , у вас все в порядке. Формы входных данных определяются автоматически, когда модель используется в `.fit` , `.evaluate` , `.predict` или при вызове модели ( `model(inputs)` ).

Вот пример:

In [ ]:
class SubclassedModel(tf.keras.Model):

  output_name = 'output_layer'

  def __init__(self):
    super(SubclassedModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(
        5, dtype=tf.dtypes.float32, name=self.output_name)

  def call(self, inputs):
    return self._dense_layer(inputs)

my_model = SubclassedModel()
# my_model.save(keras_model_path)  # ERROR! 
tf.saved_model.save(my_model, saved_model_path)